<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

In [1]:
import os
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
#from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

In [2]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

# Digitizer Signal Self Trigger

This script uses the Digitizer in a benchtop or desktop unit with self Trigger mode.<br>
    
The Signal generator is configured:<br>
DAC frequency: `2GHz`<br>

Channel 1:<br>
 - Sine waveform;  
 - amplitude 0.5V; 
 
Channel 2:<br>  
 - Triangular waveform;  
 - amplitude 0.5V;  
          
CH1 should be connected to Digitizer channel 1.<br>
CH2 should be connected to Digitizer channel 2.<br>

The Digitizer is configured:
 - Mode           : SINGLE 
 - Sample Rate    : 2GHz
 - Trigger source : CH1 (Channel 1 self trigger)
 - The Digitizer External Trigger threshold is set to 0.7V

This script uses 4 Frames with 4800 samples each.<br>

Start the Digitizer;<br>
 - wait for the status done bit goes back to 'high'.<br><br>
 
Stop the Digitizer;<br>

Read the DDR memory and plot the signals.<br>

check the proteus registers and close the script.<br>


In [3]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.0.62::5025::SOCKET'

inst = TEVisaInst(inst_addr)

resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp)

connected to: Tabor Electronics,P9484M,000002210203,1.119.0


In [4]:
# Paranoia Level (0: low, 1: normal, 2: debug)
paranoia_level = 2

# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')

if resp == 'M0': 
    dac_mode=16 
else: dac_mode=8

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get model dependant parameters:

if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    
half_dac = max_dac / 2.0    
    
print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':SOUR:FREQ 2.0e9')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

Tabor Electronics,P9484M,000002210203,1.119.0
Model: P9484M 
DAC 16 bits
0, no error


## Configure Generator

In [5]:
# data-type code: 'B' = uint8, 'H' = uint16
wav_dat_type = 'H' if dac_mode == 16 else 'B'


# Build two AWG segment that will be used as innput for the digitizer
cycleLen = 1024
numCycles = 8
segLen = cycleLen * numCycles

# Build waveform (sine wave)
x = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen, endpoint=False)

y = (np.sin(x) + 1.0) * half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
y = y.astype(data_type)

# download it to segment 2 of channel 1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEF 1,' + str(segLen), paranoia_level)
inst.send_scpi_cmd(':TRAC:SEL 1')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 1')
inst.send_scpi_cmd(':SOUR:VOLT 0.5')
inst.send_scpi_cmd(':OUTP ON')
# download the waveform to the selected segment
cmd = '*OPC?; :TRAC:DATA'
inst.write_binary_data(cmd, y)
    
    
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

y = np.sin(x)
y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
y = np.round(y)
y = np.clip(y, 0, max_dac)
y = y.astype(data_type)

# download it to segment 2 of channel 1
inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEF 2,' + str(segLen))
inst.send_scpi_cmd(':TRAC:SEL 2')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 2')
inst.send_scpi_cmd(':SOUR:VOLT 0.5')
inst.send_scpi_cmd(':OUTP ON')

# download the waveform to the selected segment
cmd = '*OPC?; :TRAC:DATA'
inst.write_binary_data(cmd, y)


resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error
0, no error


## Setup Digitizer

In [7]:
# Setup the digitizer in two-channels mode
inst.send_scpi_cmd(':DIG:MODE SING')
inst.send_scpi_cmd(':DIG:FREQ 2000e6')

# Enable capturing data from channel 1
inst.send_scpi_cmd(':DIG:CHAN CH1')
inst.send_scpi_cmd(':DIG:CHAN:STAT ENAB')
# Select the external-trigger as start-capturing trigger:
inst.send_scpi_cmd(':DIG:TRIG:SOUR CH1')
inst.send_scpi_cmd(':DIG:TRIG:SELF:LEV -0.1')


# Allocate four frames of 4800 samples
numframes, framelen = 4, 4800
cmd = ':DIG:ACQuire:FRAM:DEF {0},{1}'.format(numframes, framelen)
inst.send_scpi_cmd(cmd)

# Select the frames for the capturing 
# (all the four frames in this example)
capture_first, capture_count = 1, numframes
cmd = ":DIG:ACQ:FRAM:CAPT {0},{1}".format(capture_first, capture_count)
inst.send_scpi_cmd(cmd)

# Clean memory 
inst.send_scpi_cmd(':DIG:ACQ:ZERO:ALL')

resp =inst.send_scpi_query(':SYST:ERR?')
print(resp)
print("Set Digitizer: DUAL mode; internal Trigger")


0, no error
Set Digitizer: DUAL mode; internal Trigger


## Acquire signals into Memory

In [8]:
# Stop the digitizer's capturing machine (to be on the safe side)
inst.send_scpi_cmd(':DIG:INIT OFF')

# Start the digitizer's capturing machine
inst.send_scpi_cmd(':DIG:INIT ON')

print('Waiting for status done bit ..')
loopcount = 0
done = 0
while done == 0:
    resp = inst.send_scpi_query(":DIG:ACQ:FRAM:STATus?")
    resp_items = resp.split(',')
    done = int(resp_items[1])
    print("{0}. {1}".format(done, resp_items))
    loopcount += 1
    if loopcount == 10:
        print("No Trigger was detected")
        done = 1  
print("Capture Done")

# Stop the digitizer's capturing machine (to be on the safe side)
inst.send_scpi_cmd(':DIG:INIT OFF')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
print("Digitizer: Finish Acquisition")

Waiting for status done bit ..
1. ['1', '1', '0', '4']
Capture Done
0, no error
Digitizer: Finish Acquisition


## Read all frames from Memory

In [9]:
# Choose which frames to read (all in this example)
inst.send_scpi_cmd(':DIG:DATA:SEL ALL')

# Choose what to read 
# (only the frame-data without the header in this example)
inst.send_scpi_cmd(':DIG:DATA:TYPE FRAM')

# Get the total data size (in bytes)
resp = inst.send_scpi_query(':DIG:DATA:SIZE?')
num_bytes = np.uint32(resp)
print('Total size in bytes: ' + resp)
print()

# Read the data that was captured by channel 1:
inst.send_scpi_cmd(':DIG:CHAN:SEL 1')

wavlen = num_bytes // 2

wav1 = np.zeros(wavlen, dtype=np.uint16)

rc = inst.read_binary_data(':DIG:DATA:READ?', wav1, num_bytes)

# Read the data that was captured by channel 2:
inst.send_scpi_cmd(':DIG:CHAN:SEL 2')

wavlen = num_bytes // 2

wav2 = np.zeros(wavlen, dtype=np.uint16)

rc = inst.read_binary_data(':DIG:DATA:READ?', wav2, num_bytes)

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
print("read data from DDR")



Total size in bytes: 38400

223, settings conflict
read data from DDR


## Plot Signals

In [10]:
# Plot the samples
totlen = numframes * framelen
x = range(numframes * framelen)
plt.figure(1)
ax1 = plt.subplot(111)
ax1.set_xticks(np.arange(0, totlen, framelen))
ax1.grid(True)
ax1.plot(x, wav1)
ax1.set_ylabel('CH1')


plt.show()


<IPython.core.display.Javascript object>

## End

In [ ]:
# close connection
inst.close
